In [1]:
import ast
import pandas as pd
import numpy as np

In [2]:
# read csv files
user_data = pd.read_csv("./user_data.csv")
user_messages = pd.read_csv("./user_messages.csv")
ads_data = pd.read_csv("./ads_data.csv")

In [3]:
# see each data frame and their infromation
user_data.head()

,event_time,user_id,event,channel,user_lat,user_long,origin,ad_id,images_count,ad_impressions,ad_views,ad_messages
0,2017-05-19 10:47:22,541,view,ios,-34.600439,-58.514031,browse_search,2426321,6.0,0.0,4.0,0.0
1,2017-06-10 10:18:12,501,view,android,-34.818047,-58.356583,browse_search,2746814,5.0,152.0,85.0,1.0
2,2017-06-11 00:44:57,559,view,android,-34.688454,-58.333435,browse,1123948,4.0,899.0,22.0,1.0
3,2017-05-22 02:36:06,637,first_message,android,-34.548401,-58.485168,search,2273498,NaN,0.0,21.0,2.0
4,2017-06-08 15:45:35,509,view,android,-34.582088,-58.405041,home,2722493,3.0,1529.0,132.0,3.0


In [4]:
# ads data
ads_data.head()

,ad_id,category_id,seller_id,creation_time,title,description,price,lat,long,source,enabled
0,1344,800,1157,2015-11-29 13:05:26,Blackberry curve,"Funcionando, es para personal el único problem...",300.0,NaN,NaN,android,0
1,3916,815,59,2015-12-16 09:09:25,Remera marca premium alemana,Casi nuevo y sin uso. Lo compre online y me qu...,150.0,NaN,NaN,android,1
2,11738,806,12298,2016-01-19 18:38:24,Juego de sillones estilo campo,Divino juego de sillones de diseño campo,1500.0,NaN,NaN,android,1
3,14939,815,1349,2016-01-30 23:47:30,Zapatillas,Zapatillas nro 39 dos usos,250.0,NaN,NaN,android,0
4,16820,815,16365,2016-02-05 10:22:29,Vestido de seda negro,Vestido importado de usa. Muy fino y elegante ...,500.0,NaN,NaN,android,0


In [5]:
user_count = user_data.user_id.value_counts().reset_index()
user_count.columns = ['user_id','user_count']

user_ad_count = user_data[['user_id','ad_id','origin']].groupby(['user_id','ad_id']).agg('count').reset_index()
user_ad_count.columns = ['user_id','ad_id','usr_ad_count']

user_ad_unique = user_data[['user_id','ad_id']].groupby(['user_id']).ad_id.apply(lambda x: len(x.unique())).reset_index()
user_ad_unique.columns = ['user_id','usr_ad_unique_count']

In [6]:
ads_user = user_data[['ad_id']].merge(ads_data[['ad_id','category_id']],how = 'left',on= 'ad_id')

In [7]:
ads_count_in_user = ads_user.groupby('ad_id').agg('count').reset_index()
ads_count_in_user.columns = ['ad_id','ad_count']
ads_user = ads_user.merge(ads_count_in_user,how = 'left',on = 'ad_id')
ads_user  = ads_user.sort_values(['category_id','ad_count'],ascending=False)
ads_user = ads_user.drop_duplicates()
print("fill counts of each ads for a category viewed or messaged")
print(ads_user)

fill counts of each ads for a category viewed or messaged
           ad_id  category_id  ad_count
1997     2254453          888       497
18401    2827386          888       477
27817    2740665          888       188
47710    2578937          888       181
20472    2826518          888       178
...          ...          ...       ...
2141276  1403886          362         1
2141822  2168829          362         1
2142940  1512443          362         1
2143314  2464346          362         1
2143535  2656715          362         1

[535651 rows x 3 columns]


In [8]:
top_10_add = ads_user.groupby('category_id')['ad_id'].apply(lambda x: x.tolist()[0:10]).reset_index()

In [9]:
# get category information from ads_data and put in user_data
user_data = user_data.merge(ads_data[['ad_id', 'category_id']], how='left', on='ad_id')
user_data

,event_time,user_id,event,channel,user_lat,user_long,origin,ad_id,images_count,ad_impressions,ad_views,ad_messages,category_id
0,2017-05-19 10:47:22,541,view,ios,-34.600439,-58.514031,browse_search,2426321,6.0,0.0,4.0,0.0,888
1,2017-06-10 10:18:12,501,view,android,-34.818047,-58.356583,browse_search,2746814,5.0,152.0,85.0,1.0,815
2,2017-06-11 00:44:57,559,view,android,-34.688454,-58.333435,browse,1123948,4.0,899.0,22.0,1.0,853
3,2017-05-22 02:36:06,637,first_message,android,-34.548401,-58.485168,search,2273498,NaN,0.0,21.0,2.0,815
4,2017-06-08 15:45:35,509,view,android,-34.582088,-58.405041,home,2722493,3.0,1529.0,132.0,3.0,815
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143884,2017-06-11 03:14:26,13717,view,android,-34.625469,-58.498650,search,456667,1.0,54.0,70.0,3.0,806
2143885,2017-05-26 00:21:56,13820,view,ios,-34.630888,-58.382091,home,2700866,2.0,0.0,44.0,0.0,888
2143886,2017-05-27 05:00:26,14492,view,android,-34.593098,-58.405254,home,2164674,1.0,0.0,1.0,0.0,815
2143887,2017-05-29 22:57:32,14558,view,ios,-34.501714,-58.560002,search,2619114,7.0,0.0,68.0,6.0,800


In [10]:
user_cat_ads_list = user_data.groupby(['user_id','category_id'])['ad_id'].apply(lambda x: np.unique(x.tolist())).reset_index()

# function to explode user_cat_ads_list data frame
def explode_user_cat_ads_data(df):
    ads = []
    cat_ids = []
    user_ids = []
    for x in range(0, df.shape[0]):
        ad_list = df.ad_id.values[x]
        user_id = df.user_id[x]
        cat_id  = df.category_id[x]

        for ad in ad_list:
            user_ids.append(user_id)
            cat_ids.append(cat_id)
            ads.append(ad)
    return pd.DataFrame({'user_id':user_ids,'category_id':cat_ids,'ad_id':ads})

In [11]:
# prepare train data based on user_data and user_message. 
# all combination in user_message treated as true means target = 1
# all combination user_data other than user_message can assume as user not interested so target = 0
train_user_data = explode_user_cat_ads_data(user_cat_ads_list)
train_user_data = train_user_data.drop_duplicates()
train_user_data['target'] = 0
print(train_user_data)

         user_id  category_id    ad_id  target
0              1          800   337350       0
1              1          800  1227188       0
2              1          800  1232646       0
3              1          800  1272760       0
4              1          800  1379235       0
...          ...          ...      ...     ...
1614534    15065          888  2609234       0
1614535    15065          888  2714279       0
1614536    15065          888  2714667       0
1614537    15065          888  2798540       0
1614538    15065          888  2850173       0

[1614539 rows x 4 columns]


In [12]:
def explode_user_message_data(df):
    ads = []
    cat_ids = []
    user_ids = []
    for x in range(0, df.shape[0]):
        ads_list = df.ads.values[x]
        user_id = df.user_id[x]
        cat_id  = df.category_id[x]
        ads_list = ast.literal_eval(ads_list)

        for ad in ads_list:
            user_ids.append(user_id)
            cat_ids.append(cat_id)
            ads.append(ad)
    return pd.DataFrame({'user_id':user_ids,'category_id':cat_ids,'ad_id':ads})

In [13]:
train_user_messages = explode_user_message_data(user_messages)
train_user_messages = train_user_messages.drop_duplicates()
train_user_messages['target'] = 1

In [14]:
train_final = train_user_messages.merge(train_user_data,on = [ 'user_id','category_id','ad_id'],how = 'outer')
train_final

,user_id,category_id,ad_id,target_x,target_y
0,1,859,1806476,1.0,NaN
1,3,800,2131700,1.0,NaN
2,3,800,2734107,1.0,NaN
3,3,800,2877209,1.0,NaN
4,4,815,2883211,1.0,NaN
...,...,...,...,...,...
1634094,15065,888,2609234,NaN,0.0
1634095,15065,888,2714279,NaN,0.0
1634096,15065,888,2714667,NaN,0.0
1634097,15065,888,2798540,NaN,0.0


In [15]:
train_final.loc[train_final.target_x ==1,'target'] = 1
train_final.loc[pd.isnull(train_final.target_x) & (train_final.target_y == 0),'target'] = 0
train_final

,user_id,category_id,ad_id,target_x,target_y,target
0,1,859,1806476,1.0,NaN,1.0
1,3,800,2131700,1.0,NaN,1.0
2,3,800,2734107,1.0,NaN,1.0
3,3,800,2877209,1.0,NaN,1.0
4,4,815,2883211,1.0,NaN,1.0
...,...,...,...,...,...,...
1634094,15065,888,2609234,NaN,0.0,0.0
1634095,15065,888,2714279,NaN,0.0,0.0
1634096,15065,888,2714667,NaN,0.0,0.0
1634097,15065,888,2798540,NaN,0.0,0.0


In [16]:
test_data = pd.read_csv("./user_messages_test.csv")
test_final= test_data.merge(train_user_data, on = ['user_id','category_id'], how = 'left')
test_final  = test_final.drop('target',axis =1)

In [17]:
test_final.replace(np.nan, -1, inplace= True)

In [18]:
test_no_infor = test_final.loc[test_final.ad_id == -1,['user_id','category_id']]
test_no_infor = test_no_infor.merge(top_10_add, on = 'category_id',how = 'left')
print(test_no_infor)

      user_id  category_id                                              ad_id
0           2          800  [2814436, 2806603, 2816630, 2816336, 2802819, ...
1          14          811  [2626729, 2764838, 1928465, 2753896, 2597250, ...
2          20          800  [2814436, 2806603, 2816630, 2816336, 2802819, ...
3          49          800  [2814436, 2806603, 2816630, 2816336, 2802819, ...
4          51          815  [2234932, 2434247, 2796911, 2812937, 2812849, ...
...       ...          ...                                                ...
2401    15046          806  [2804693, 2809690, 26400, 2825119, 2788868, 27...
2402    15058          800  [2814436, 2806603, 2816630, 2816336, 2802819, ...
2403    15059          806  [2804693, 2809690, 26400, 2825119, 2788868, 27...
2404    15060          815  [2234932, 2434247, 2796911, 2812937, 2812849, ...
2405    15064          881  [2794985, 2816864, 2817712, 2840833, 2519508, ...

[2406 rows x 3 columns]


In [19]:
def explode_test_no_info(df):
    k = []
    us = []
    ca = []
    for enu in range(0,df.shape[0]):
        i = df.ad_id.values[enu]
        user = df.user_id[enu]
        cat  = df.category_id[enu]
        #a = ast.literal_eval(i)
        for j in i:
            us.append(user)
            ca.append(cat)
            k.append(j)
    return pd.DataFrame({'user_id':us,'category_id':ca,'ad_id':k})

In [20]:
test_no_infor = explode_test_no_info(test_no_infor)
test_no_infor = test_no_infor.drop_duplicates()

In [21]:
test_final = pd.concat((test_final[['ad_id','category_id','user_id']],test_no_infor[['ad_id','category_id','user_id']]))
test_final = test_final.loc[test_final.ad_id != -1]
test_final = test_final.drop_duplicates()
test_final.shape
print(test_final)

           ad_id  category_id  user_id
1       158425.0          815        6
2       193203.0          815        6
3       291617.0          815        6
4       299621.0          815        6
5       353091.0          815        6
...          ...          ...      ...
24055  2727695.0          881    15064
24056  2662331.0          881    15064
24057  2767340.0          881    15064
24058  2846976.0          881    15064
24059  2688815.0          881    15064

[406793 rows x 3 columns]


In [22]:
test_final = test_final[['user_id','category_id','ad_id']]
test_final

,user_id,category_id,ad_id
1,6,815,158425.0
2,6,815,193203.0
3,6,815,291617.0
4,6,815,299621.0
5,6,815,353091.0
...,...,...,...
24055,15064,881,2727695.0
24056,15064,881,2662331.0
24057,15064,881,2767340.0
24058,15064,881,2846976.0


In [23]:
train_final = train_final.drop(['target_y','target_x'],axis=1)
train_final

,user_id,category_id,ad_id,target
0,1,859,1806476,1.0
1,3,800,2131700,1.0
2,3,800,2734107,1.0
3,3,800,2877209,1.0
4,4,815,2883211,1.0
...,...,...,...,...
1634094,15065,888,2609234,0.0
1634095,15065,888,2714279,0.0
1634096,15065,888,2714667,0.0
1634097,15065,888,2798540,0.0


In [24]:
#add other informarion from user data to train the model, and do data processing
#ads and user data again
user_data = pd.read_csv("./user_data.csv")
ads_data = pd.read_csv("./ads_data.csv")

train_final = train_final.merge(user_data,on = ['user_id','ad_id'] ,how = 'left')
test_final = test_final.merge(user_data,on = ['user_id','ad_id'] ,how = 'left')

In [25]:
train_final['event_time'] = pd.to_datetime(train_final.event_time)
test_final['event_time'] = pd.to_datetime(test_final.event_time)

train_final['year'] = train_final.event_time.dt.year
train_final['month'] = train_final.event_time.dt.month
train_final['dayofyear'] = train_final.event_time.dt.dayofyear
train_final['dayofweek'] = train_final.event_time.dt.dayofweek
train_final['day'] = train_final.event_time.dt.day

test_final['year'] = test_final.event_time.dt.year
test_final['month'] = test_final.event_time.dt.month
test_final['dayofyear'] = test_final.event_time.dt.dayofyear
test_final['dayofweek'] = test_final.event_time.dt.dayofweek
test_final['day'] = test_final.event_time.dt.day

In [26]:
# put ads information from ads table
train_final = train_final.merge(ads_data,how='left',on=['ad_id', 'category_id'])
test_final = test_final.merge(ads_data,how='left',on=['ad_id', 'category_id'])

In [27]:
# add user having origin, ads and category
train_final = train_final.merge(user_count,how='left',on=['user_id'])
test_final = test_final.merge(user_count,how='left',on=['user_id'])

train_final = train_final.merge(user_ad_count,how='left',on=['user_id','ad_id'])
test_final = test_final.merge(user_ad_count,how='left',on=['user_id','ad_id'])

train_final = train_final.merge(user_ad_unique,how='left',on=['user_id'])
test_final = test_final.merge(user_ad_unique,how='left',on=['user_id'])

In [28]:
train_final = train_final.drop(['creation_time','lat', 'long','channel', u'user_lat', u'user_long','title', 'description'],axis =1)
test_final = test_final.drop(['creation_time','lat', 'long','channel', u'user_lat', u'user_long','title', 'description'],axis =1)

In [29]:
test_final['enabled'].replace(np.nan, 1, inplace = True)

In [30]:
print(test_final)
print(train_final)

        user_id  category_id      ad_id          event_time event  \
0             6          815   158425.0 2017-05-25 00:48:19  view   
1             6          815   193203.0 2017-05-23 22:11:22  view   
2             6          815   291617.0 2017-05-25 00:37:57  view   
3             6          815   299621.0 2017-05-25 00:48:57  view   
4             6          815   353091.0 2017-05-25 00:50:16  view   
...         ...          ...        ...                 ...   ...   
545614    15064          881  2727695.0                 NaT   NaN   
545615    15064          881  2662331.0                 NaT   NaN   
545616    15064          881  2767340.0                 NaT   NaN   
545617    15064          881  2846976.0                 NaT   NaN   
545618    15064          881  2688815.0                 NaT   NaN   

               origin  images_count  ad_impressions  ad_views  ad_messages  \
0       browse_search           5.0             0.0       5.0          0.0   
1              

In [39]:
#pre processing label encoder
from sklearn.preprocessing import  LabelEncoder
text_columns = []
for f in test_final.columns:
    if train_final[f].dtype== 'object':
        text_columns.append(f)            
        lbl = LabelEncoder()
        lbl.fit(list(train_final[f].values) + list(test_final[f].values))
        train_final[f] = lbl.transform(list(train_final[f].values))
        test_final[f] = lbl.transform(list(test_final[f].values))

In [40]:
train_final

,event,origin,images_count,ad_impressions,ad_views,ad_messages,year,month,dayofyear,dayofweek,day,seller_id,price,source,enabled,user_count,usr_ad_count,usr_ad_unique_count
0,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,294873.0,1200.0,0,1.0,27.0,-1.0,25.0
1,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,275522.0,1400.0,0,1.0,-1.0,-1.0,-1.0
2,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,316586.0,1500.0,0,1.0,-1.0,-1.0,-1.0
3,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,429006.0,800.0,0,1.0,-1.0,-1.0,-1.0
4,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,386465.0,600.0,0,1.0,638.0,-1.0,483.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163453,2,8,4.0,14.0,24.0,1.0,2017.0,6.0,157.0,1.0,6.0,451539.0,1000.0,0,0.0,145.0,6.0,117.0
2163454,2,3,4.0,32.0,26.0,2.0,2017.0,6.0,158.0,2.0,7.0,451539.0,1000.0,0,0.0,145.0,6.0,117.0
2163455,0,4,3.0,1038.0,95.0,8.0,2017.0,6.0,158.0,2.0,7.0,614300.0,350.0,0,0.0,145.0,2.0,117.0
2163456,2,4,3.0,1038.0,95.0,8.0,2017.0,6.0,158.0,2.0,7.0,614300.0,350.0,0,0.0,145.0,2.0,117.0


In [41]:
test_final

,event,origin,images_count,ad_impressions,ad_views,ad_messages,year,month,dayofyear,dayofweek,day,seller_id,price,source,enabled,user_count,usr_ad_count,usr_ad_unique_count
0,2,1,5.0,0.0,5.0,0.0,2017.0,5.0,145.0,3.0,25.0,54896,80.0,0,0,806.0,1.0,465.0
1,2,8,2.0,0.0,10.0,0.0,2017.0,5.0,143.0,1.0,23.0,128675,530.0,0,1,806.0,1.0,465.0
2,2,1,4.0,0.0,9.0,1.0,2017.0,5.0,145.0,3.0,25.0,184675,500.0,0,1,806.0,1.0,465.0
3,2,1,8.0,0.0,2.0,0.0,2017.0,5.0,145.0,3.0,25.0,188060,200.0,0,0,806.0,1.0,465.0
4,2,1,1.0,0.0,1.0,0.0,2017.0,5.0,145.0,3.0,25.0,217530,200.0,0,0,806.0,1.0,465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545614,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,194180,1300.0,0,0,-1.0,-1.0,-1.0
545615,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,524741,900.0,0,1,-1.0,-1.0,-1.0
545616,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,616839,130.0,0,0,-1.0,-1.0,-1.0
545617,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,439914,210.0,0,1,-1.0,-1.0,-1.0


In [44]:
target = train_final.target.values
test_f = test_final.copy()

train_final = train_final.drop(['target','event_time','user_id', 'category_id', 'ad_id'],axis =1)
test_final = test_final.drop(['event_time','user_id', 'category_id', 'ad_id'],axis =1)

AttributeError: 'DataFrame' object has no attribute 'target'

In [45]:
train_final.replace(np.nan,-1,inplace=True)
test_final.replace(np.nan,-1,inplace=True)

In [46]:
train_final.columns

Index(['event', 'origin', 'images_count', 'ad_impressions', 'ad_views',
       'ad_messages', 'year', 'month', 'dayofyear', 'dayofweek', 'day',
       'seller_id', 'price', 'source', 'enabled', 'user_count', 'usr_ad_count',
       'usr_ad_unique_count'],
      dtype='object')

In [47]:
test_final.columns

Index(['event', 'origin', 'images_count', 'ad_impressions', 'ad_views',
       'ad_messages', 'year', 'month', 'dayofyear', 'dayofweek', 'day',
       'seller_id', 'price', 'source', 'enabled', 'user_count', 'usr_ad_count',
       'usr_ad_unique_count'],
      dtype='object')

In [48]:
!pip install xgboost

In [49]:
import xgboost as xgb
from sklearn.metrics import  roc_auc_score
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0):
    params = {}
    params["objective"] = "binary:logistic"
    params['eval_metric'] = 'auc'
    params["eta"] = 0.01
    params["min_child_weight"] = 1
    params["subsample"] = 0.8
    params["colsample_bytree"] = 0.3
    params["silent"] = 1
    params["max_depth"] = 6
    params["seed"] = seed_val
    num_rounds = 1500 #2500

    plst = list(params.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds= 500)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    if feature_names:
        create_feature_map(feature_names)
        model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
        importance = model.get_fscore(fmap='xgb.fmap')
        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
        imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
        imp_df.to_csv("imp_feat.txt", index=False)

    pred_test_y = model.predict(xgtest)
        
    if test_y is not None:
        loss = roc_auc_score(test_y, pred_test_y)
        print (loss)
        return pred_test_y, loss
    else:
        return pred_test_y

In [50]:
X = np.array(train_final)
Y = np.array(test_final)
k = runXGB(X, target, Y)

test_final

,event,origin,images_count,ad_impressions,ad_views,ad_messages,year,month,dayofyear,dayofweek,day,seller_id,price,source,enabled,user_count,usr_ad_count,usr_ad_unique_count
0,2,1,5.0,0.0,5.0,0.0,2017.0,5.0,145.0,3.0,25.0,54896,80.0,0,0,806.0,1.0,465.0
1,2,8,2.0,0.0,10.0,0.0,2017.0,5.0,143.0,1.0,23.0,128675,530.0,0,1,806.0,1.0,465.0
2,2,1,4.0,0.0,9.0,1.0,2017.0,5.0,145.0,3.0,25.0,184675,500.0,0,1,806.0,1.0,465.0
3,2,1,8.0,0.0,2.0,0.0,2017.0,5.0,145.0,3.0,25.0,188060,200.0,0,0,806.0,1.0,465.0
4,2,1,1.0,0.0,1.0,0.0,2017.0,5.0,145.0,3.0,25.0,217530,200.0,0,0,806.0,1.0,465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545614,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,194180,1300.0,0,0,-1.0,-1.0,-1.0
545615,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,524741,900.0,0,1,-1.0,-1.0,-1.0
545616,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,616839,130.0,0,0,-1.0,-1.0,-1.0
545617,1,5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,439914,210.0,0,1,-1.0,-1.0,-1.0


In [41]:
train_final.columns

Index(['event', 'origin', 'images_count', 'ad_impressions', 'ad_views',
       'ad_messages', 'year', 'month', 'dayofyear', 'dayofweek', 'day',
       'seller_id', 'price', 'source', 'enabled', 'user_count', 'usr_ad_count',
       'usr_ad_unique_count'],
      dtype='object')

In [55]:
test_f['prob'] = k
test_f.loc[test_final.enabled ==0,'prob'] =0
test_f

,user_id,category_id,ad_id,event_time,event,origin,images_count,ad_impressions,ad_views,ad_messages,...,dayofweek,day,seller_id,price,source,enabled,user_count,usr_ad_count,usr_ad_unique_count,prob
0,6,815,158425.0,2017-05-25 00:48:19,2,1,5.0,0.0,5.0,0.0,...,3.0,25.0,54896,80.0,0,0,806.0,1.0,465.0,0.000000
1,6,815,193203.0,2017-05-23 22:11:22,2,8,2.0,0.0,10.0,0.0,...,1.0,23.0,128675,530.0,0,1,806.0,1.0,465.0,0.000323
2,6,815,291617.0,2017-05-25 00:37:57,2,1,4.0,0.0,9.0,1.0,...,3.0,25.0,184675,500.0,0,1,806.0,1.0,465.0,0.000623
3,6,815,299621.0,2017-05-25 00:48:57,2,1,8.0,0.0,2.0,0.0,...,3.0,25.0,188060,200.0,0,0,806.0,1.0,465.0,0.000000
4,6,815,353091.0,2017-05-25 00:50:16,2,1,1.0,0.0,1.0,0.0,...,3.0,25.0,217530,200.0,0,0,806.0,1.0,465.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545614,15064,881,2727695.0,NaT,1,5,NaN,NaN,NaN,NaN,...,NaN,NaN,194180,1300.0,0,0,NaN,NaN,NaN,0.000000
545615,15064,881,2662331.0,NaT,1,5,NaN,NaN,NaN,NaN,...,NaN,NaN,524741,900.0,0,1,NaN,NaN,NaN,0.999953
545616,15064,881,2767340.0,NaT,1,5,NaN,NaN,NaN,NaN,...,NaN,NaN,616839,130.0,0,0,NaN,NaN,NaN,0.000000
545617,15064,881,2846976.0,NaT,1,5,NaN,NaN,NaN,NaN,...,NaN,NaN,439914,210.0,0,1,NaN,NaN,NaN,0.999971


In [56]:
test_f = test_f[['user_id','category_id','ad_id','prob']].drop_duplicates()

In [57]:
test_ad_id_list = test_f.groupby(['category_id','user_id'])['ad_id'].apply(lambda x: x.tolist()).reset_index()
test_prob_list = test_f.groupby(['category_id','user_id'])['prob'].apply(lambda x: x.tolist()).reset_index()

In [58]:
test_ad_id_list['prob'] = test_prob_list['prob']

In [59]:
#sort based on the high to low probability
def top10forEachCatWithHighProb(x):
    val = []
    ad_id = x[0]
    prob =  x[1]
    a = np.argsort(prob)[-10:][::-1]
    for i in a:
        val.append(int(ad_id[i]))
    return val
    
test_ad_id_list['ads'] = test_ad_id_list[['ad_id','prob','category_id']].apply(top10forEachCatWithHighProb,axis =1)

,category_id,user_id,ad_id,prob,ads
0,362,32,"[22019.0, 22019.0, 101978.0, 234695.0, 265164....","[0.0002419164084130898, 0.0011678014416247606,...","[2155129, 430569, 2192663, 2451503, 2415961, 1..."
1,362,42,"[222756.0, 335069.0, 579051.0, 633700.0, 63370...","[0.0, 0.0029604830779135227, 0.004887063521891...","[2684942, 2680584, 1642256, 2230578, 2687448, ..."
2,362,66,"[1635665.0, 1635665.0, 1782977.0, 2069503.0, 2...","[0.00841978657990694, 0.005475614685565233, 0....","[1635665, 1635665, 2833390, 2833390, 2833390, ..."
3,362,182,"[219720.0, 1524344.0, 2297932.0]","[0.0, 0.0007145960698835552, 0.000862761866301...","[2297932, 1524344, 219720]"
4,362,191,"[5073.0, 44852.0, 384321.0, 1564248.0, 1704736...","[0.0033872374333441257, 0.0037937455344945192,...","[1937355, 1704736, 2806363, 2377472, 44852, 50..."
...,...,...,...,...,...
10502,888,14928,"[568919.0, 571526.0, 833936.0, 1317639.0, 1333...","[0.0018843853613361716, 0.0, 0.0, 0.0, 0.0, 0....","[1897754, 2799018, 2427296, 2720400, 568919, 2..."
10503,888,14965,"[614736.0, 656506.0, 670515.0, 676058.0, 67688...","[0.002899731043726206, 0.0, 0.0, 0.0, 0.0, 0.0...","[1980241, 2533332, 2431912, 2482226, 1929653, ..."
10504,888,14969,"[493332.0, 547875.0, 753459.0, 1002787.0, 1019...","[0.0, 0.001346529577858746, 0.0023784732911735...","[2316472, 2316472, 2155999, 1019648, 1546624, ..."
10505,888,15021,"[59771.0, 529473.0, 560006.0, 623556.0, 139949...","[0.0, 0.0, 0.000927009095903486, 0.0, 0.002454...","[1399494, 1399494, 1399494, 2527821, 2504388, ..."


In [61]:
top_10_add = ads_user.groupby('category_id')['ad_id'].apply(lambda x: x.tolist()[0:10]).reset_index()
top_10_add = top_10_add.set_index('category_id')['ad_id'].to_dict()

In [65]:
# what if ads is no more enabled or available
dt = ads_data.set_index('ad_id')['enabled'].to_dict()

In [66]:
# top 10 ads of each category and enabled but are not in final list
# also length is less than 10 then add those in the list
def fillRemaining(x):
    
    x0 = x[0]
    x1 = x[1]
    k = len(x1)
    if len(x1)<10:
        for i in top_10_add[x0]:
            if (i not in x1) & (dt[i]==1):
                x1.append(i)
                if len(x1) ==10:
                    break
    
    return x1[:10]

In [67]:
test_ad_id_list['ads'] = test_ad_id_list[['category_id','ads']].apply(fillRemaining,axis =1)

In [68]:
# save to csv file
test_ad_id_list[['user_id','category_id','ads']].to_csv('ads_recommendation.csv',index = False)